# Non-thermal radiation with naima
In this jupyter notebook we will learn how to use the package naima to build non-thermal emission models. In the previous lectures we made many assumptions: mono-energetic electrons and photons and Thomson limit in the case of Inverse Compton emission. The naima modul can be used for any spectra of particles and photons. It also allows to fit the model to observational data.
You can read more on naima on this web page:
https://naima.readthedocs.io

This Notebook contains 3 exercises for a total of 9 marks. Please submit on Ulwazi.

## Pre-requisites
As usual, we need to import the modules.

In [ ]:
import numpy as np

import matplotlib.pyplot as plt

from astropy import units as u
from astropy import constants as c

from numpy import sqrt

From naima we will import only the parts that we will make use of.

In [ ]:
from naima.models import Synchrotron, InverseCompton, PionDecay
from naima.models import PowerLaw, ExponentialCutoffPowerLaw

We will look at the total energy ouput of a hypothetical source. We will note yet deal with the flux which can be measured on Earth. So we will ignore the distance to the source. We will do so by using a distance of 0.

In [ ]:
d = 0 * u.kpc

## Particle Spectra
For our example we will assume a particle spectrum following a power law with index 2.5, with an amplitude $A$ at a reference energy of 1 TeV and an exponential cut off at an energy of 10 TeV. A power law with an exponential cut-off is given by
$$
f(E) = A \left( \frac{E}{E_0} \right)^{-\Gamma} \times \exp{\left( -\left(\frac{E}{E_\mathsf{cut}}\right)^\alpha \right)} 
$$
We will use $\alpha = 1$ throughout this notebook. This is the default value, we simply ignore it.

In [ ]:
A = 1e45/u.TeV
E_0 = 1 * u.TeV
Gamma = 2.5
E_cut = 10 * u.TeV

In [ ]:
PL = ExponentialCutoffPowerLaw(A, E_0, Gamma, E_cut)

We want to take a look at the spectrum in an energy range from 1 MeV to 100 TeV. (We should go down to 0.511 MeV as this is the electron rest energy. But this does not make a big difference).

In [ ]:
energies = np.logspace(-6,2,801) * u.TeV

In [ ]:
N = PL(energies)

Let's make a plot of it:

In [ ]:
plt.loglog(energies, N)

plt.xlabel('E [{}]'.format(energies.unit))
plt.ylabel('dN/dE [{}]'.format(N.unit))

You see that above $\approx 1\,\mathsf{TeV}$ the spectrum sharply drops off from the straight power law.

Note that this power law object describes only the shape of the spectrum. It tells us how many particles are found at which energy. It does not make any claim on the type of the particle. It can be electrons, protons, photons or anything else.

We can get the total energy stored in these particles by integrating over the number of particles (N) multiplied with the energy (from array energies):

In [ ]:
E_tot = np.trapz(N*energies, energies)

In [ ]:
E_tot.to(u.erg)

## Synchrotron Radiation
Naima provides a model for synchrotron radiation. We will use the magnetic field as before.

In [ ]:
B = 1e-8 * u.T

We will create a synchrotron model object. The shape of the electron spectrum is given by the power law with cut-off as defined above. We also need to indicate the electron's minimal energy of 511 keV, as this is lower than the default value of 1 GeV.

In [ ]:
sync = Synchrotron(PL, B, Eemin = 511*u.keV)

Note that this object has additional parameters. The maximum energy can be given as well (parameter Eemax) and the default value is 510 TeV. The number of points can be changed ro reduce computing time or increase accuracy (parameter nEed, default value 100).

This object allows to calculate the total energy in electrons. This should correspond to our calculation above.

In [ ]:
sync.We

You can also calculate the energy in electrons in a certain energy range, let's say from 100 GeV to 1 TeV:

In [ ]:
sync.compute_We(100*u.GeV, 1*u.TeV)

Let's create an energy range for the photons. This will define the range of the electromagnetic spectrum we want to study. We will study the range from $10^{-19}\,\mathsf{TeV}$ to $100\,\mathsf{TeV}$.

In [ ]:
photon_energies = np.logspace(-19,2,211) * u.TeV

With the flux function of the sync object we can get the number of photons produced every second. We have to set the distance to 0, which we have defined as d = 0 above.

In [ ]:
flux_sync = sync.flux(photon_energies, distance = d)

In [ ]:
flux_sync

We can also use the function sed (which stands for spectral energy distribution) to get the energy flux, or with a distance of zero the luminosity of source.

In [ ]:
sed_sync = sync.sed(photon_energies, distance = d)

In [ ]:
sed_sync

You can get the SED by multiplying the flux with the square of the photon energy. Let's try this:

In [ ]:
(flux_sync*(photon_energies**2)).to(u.erg/u.s)

In [ ]:
Eflux_sync = flux_sync * photon_energies

In [ ]:
Eflux_sync

We can integrate over Eflux to get the total power in synchrotron radiation.

In [ ]:
P_sync = (np.trapz(Eflux_sync, photon_energies)).to(u.erg/u.s)

In [ ]:
P_sync

We can make a plot of the synchrotron radiation. Usually the SED is plotted in astrophysical publications.

In [ ]:
plt.loglog(photon_energies, sed_sync)

plt.title('synchrotron radiation')

plt.xlabel('$E$ [{}]'.format(photon_energies.unit))
plt.ylabel('$E^2 dN/dE$ [{}]'.format(sed_sync.unit))

plt.ylim(1e24, 1e36)

## Inverse Compton Emission
Similarly, you can also get the IC emission of these electrons. There several predefined photon fields in naima. You can also define your own photon field. For the moment we deal only with photons of the Cosmic Microwave Background (CMB).

In [ ]:
photon_fields = ['CMB']

In [ ]:
IC = InverseCompton(PL, photon_fields, Eemin = 511*u.keV)

This works exactly as the synchrotron object, so we can do the same tests as above.

In [ ]:
IC.We

In [ ]:
flux_ic = IC.flux(photon_energies, distance = d)

sed_ic = IC.sed(photon_energies, distance = d)

Eflux_ic = flux_ic * photon_energies

In [ ]:
P_IC = (np.trapz(Eflux_ic, photon_energies)).to(u.erg/u.s)

In [ ]:
P_IC

This is the power radiated from Inverse Compton scattering. This is much lower than what we have calculated for the synchrotron radiation. Keep in mind, that ususally the cooling is synchrotron dominated.

Let's try to calculate the magnetic field from the power in synchrotron and inverse Compton emission. The equation from lecture slide 12 of "Inverse Compton II" is 
$$
B = \sqrt{ \frac{u_\mathsf{rad} P_\mathsf{sync} 2 \mu_0}{P_\mathsf{IC}} }
$$
where $u_\mathsf{rad} = 0.25\,\mathsf{eV}/\mathsf{cm}^3$ is the energy density of the CMB photon field.

In [ ]:
B_measured = sqrt(0.25*u.eV/u.cm**3 * P_sync * 2 * c.mu0 / P_IC)

In [ ]:
B_measured.to(u.T)

This is the vaule we have put into our model!

Let's make a plot of the Inverse Compton emission.

In [ ]:
plt.loglog(photon_energies, sed_ic)

plt.title('Inverse Compton emission')

plt.xlabel('$E$ [{}]'.format(photon_energies.unit))
plt.ylabel('$E^2 dN/dE$ [{}]'.format(sed_ic.unit))

Note the break at lower energies. This is due to the minimum energy of 511 keV. You can test this yourself. We can safely ignore this, this is at levels that we cannot measure anyway.

## Combined Plot
We can show both emissions in one plot. This is the SED of the source.

In [ ]:
plt.loglog(photon_energies, sed_sync, label='synchrotron')
plt.loglog(photon_energies, sed_ic, label='inverse Compton off CMB')

plt.legend()
plt.xlabel('$E$ [{}]'.format(photon_energies.unit))
plt.ylabel('$E^2 dN/dE$ [{}]'.format(sed_ic.unit))

plt.ylim(1e29, 1e36)

You can also sum up the two components and show the total emission.

In [ ]:
sed_sum = sed_sync + sed_ic

In [ ]:
plt.loglog(photon_energies, sed_sum)

### Exercise
We keep the electron spectrum as it is. We want to understand what is happening when we increase or decrease the magnetic field. Plot the total emission (synchrotron and inverse Compton emission) for the cases of $10^{-7}\,\mathsf{T}$, $10^{-8}\,\mathsf{T}$ and $10^{-9}\,\mathsf{T}$.

**[3 marks]**

In [ ]:
Bs = [1e-7,1e-8,1e-9] *u.T

In [ ]:
# your code here



Expected output:


## Hadronic emission
Naima also provides a model for gamma-ray production through pion decay in inelastic proton-proton collisions. The only parameter is the target material density, we will use 1 particle per cm$^3$.

In [ ]:
n = 1./ u.cm**3

We will define a new spectrum for the protons and keep the same parameters. (We could use the same as above, but we want to be sure that we do not mix up the different particles.)

In [ ]:
PL_protons = ExponentialCutoffPowerLaw(A, E_0, Gamma, E_cut)

In [ ]:
had = PionDecay(PL_protons, n)

Similar to what was done above we can calculate the total energy in protons. As we are talking about protons now we have to call Wp:

In [ ]:
had.Wp

This is what we have put in above. But for protons this might be a bit low. For a supernova remnant for example we are expecting up to $10^{50}\,\mathsf{erg}$ which is about 10% of the explosion energy. We can set the total energy in the PionDecay object:

In [ ]:
had.set_Wp(1e50*u.erg)

In [ ]:
had.Wp

This worked. This changed the amplitude of our spectrum. Let's check:

In [ ]:
PL_protons.amplitude

Now we can do the same steps as above:

In [ ]:
flux_had = had.flux(photon_energies, distance = d)

sed_had = had.sed(photon_energies, distance = d)

Eflux_had = flux_had * photon_energies

In [ ]:
P_had = (np.trapz(Eflux_had, photon_energies)).to(u.erg/u.s)

In [ ]:
P_had

In [ ]:
plt.loglog(photon_energies, sed_had)

plt.title('hadronic emission')

plt.xlabel('$E$ [{}]'.format(photon_energies.unit))
plt.ylabel('$E^2 dN/dE$ [{}]'.format(sed_ic.unit))

plt.ylim(1e30)
plt.xlim(1e-6)

You see the characteristic turn-over at lower energies due to the pion mass. The cut-off at high energies is due to the break in the proton spectrum.

### Exercise
We keep the proton spectrum as it is. We want to understand what is happening when we increase or decrease the density. Plot the hadronic emission for the cases of $1\,\mathsf{cm}^{-3}$, $10\,\mathsf{cm}^{-3}$ and $100\,\mathsf{cm}^{-3}$.

**[3 marks]**

In [ ]:
densities = [1, 10, 100]/u.cm**3

In [ ]:
# your code here

plt.ylim(1e30)
plt.xlim(1e-6)

Expected output:


### Exercise
We keep the amplitude of the proton spectrum as it is and the density at $1\,\mathsf{cm}^{-3}$. We want to understand what is happening when we change the spectral index. Plot the hadronic emission for spectral indices of 2, 2.3 and 2.6.

**[3 marks]**

In [ ]:
n = 1/u.cm**3
SpecIndx = [2, 2.3, 2.6]

In [ ]:
# your code here

plt.ylim(1e30)
plt.xlim(1e-6)

Expected output:


## Submission

Before you submit your work you should make a few checks that everything works fine.

1. Save your notebook as a PDF (File->Download As->PDF). This document will help you debugging in the next step.
1. If PDF export does not work: You can do File->Print Preview and then print to a file.
1. Restart the kernel and rerun the entire notebook (Kernel->Restart & Run All). This will delete all variables (but not your code) and rerun the notebook in one go. If this does not go through the endthen you have to fix it. You will see at which cell the run stopped. A common mistake is using a variable that is defined only at a later stage.
1. You think you fixed everything? Redo step 2 (Kernel->Restart & Run All)

You have to download and submit 2 files, the jupyter notebook and a pdf.
- Jupyter notebook. File->Download As->Notebook (.ipynb). Save this file on your disk.
- PDF file. File->Download As->PDF. Save this file on your disk.
- If PDF export does not work. You can do File->Print Preview and then print to a file.

Please submit the two files on Ulwazi.